### Домашнее задание №5 к лекции "Основы веб-скрапинга"

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from time import sleep
from datetime import datetime
import re
from tqdm import tqdm

#### Задание 1
Вам необходимо написать функцию, которая будет основана на поиске по сайту http://habr.com. Функция в качестве параметра должна принимать список запросов для поиска (например, ['python', 'анализ данных']) и на основе материалов, попавших в результаты поиска по каждому запросу, возвращать датафрейм вида:

<дата> - <заголовок> - <ссылка на материал>
В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.

In [2]:
req = ['большие данные', 'python', 'sql', 'big data']

Так как HTML извлекать придется во каждом случае, создадим для этого вспомогательную функцию.

In [3]:
def get_html(url, params=None):
    ''' 
    Функция для получения HTML кода из url, 
    в слачае 200 ответа сайта
    '''
    html = requests.get(url, params=params)
    if html.status_code == 200:
        return html.text

In [4]:
def get_content_lite(req):
    '''
    Функия получения короткой таблицы со статьями по списку поисковых слов
    На вход принимает список
    Внутри, 
    1) для каждого слова делает запрос к сайту
    2) перебирает блоки со статьями на первой странице поиска
    3) для каждой статьи собирает дату, название, ссылку, атора
    4) добавляет конкретное поисковое слово
    5) доавляет полученную строку к датафрейму
    6) пробегает циклом по всем словам из переданного списка
    7) удаляет дубли по столбцу со ссылками
    работает только для первой страницы поиска
    '''
    url = 'https://habr.com/ru/search/'
    data = pd.DataFrame()
    for word in req:
        params = {'q': word}
        html = get_html(url, params=params) # используем вспомогательную функцию для получения html
        soup = BeautifulSoup(html, 'html.parser')
        articles = soup.find('div', class_='tm-articles-list').find_all('div', 'tm-article-snippet')
        for item in tqdm(articles):
            sleep(0.1)
            date = pd.to_datetime(item.find('time').get('datetime')).strftime('%Y-%m-%d %H:%M') 
            title = item.find('a', 'tm-article-snippet__title-link').text
            link = 'https://habr.com' + item.find('a', 'tm-article-snippet__title-link').get('href')
            author = item.find('span', 'tm-user-info__user').text.strip()
            row = {'word':word, 
                   'date':date, 
                   'title':title, 
                   'link':link, 
                   'author':author}
            data = data.append(row, ignore_index=True)
    data.drop_duplicates(subset='link', inplace=True)
    return data

In [5]:
%%time
content_lite = get_content_lite(req)

100%|██████████| 19/19 [00:02<00:00,  8.78it/s]

CPU times: user 1.13 s, sys: 91.8 ms, total: 1.23 s
Wall time: 11.8 s


In [6]:
content_lite.sort_values(by='date', ascending=False).head(10)

,word,date,title,link,author
39,sql,2022-05-20 11:19,Do it yourself: JIT компиляция SQL в Tarantool,https://habr.com/ru/company/vk/blog/666370/,CuriousGeorgiy
35,python,2022-05-18 15:33,Слёрм запускает 3-дневный интенсив по Python д...,https://habr.com/ru/company/southbridge/news/t...,edeshina
74,big data,2022-05-04 08:44,Data-Science-процессы: Jupyter Notebook для пр...,https://habr.com/ru/company/vk/blog/662734/,Olga_Mokshina
37,python,2022-04-22 11:42,TechnoMeetsPython. Онлайн митап о Python-разра...,https://habr.com/ru/news/t/662437/,technokratiya
13,большие данные,2022-04-07 14:23,17 лучших инструментов и технологий для работы...,https://habr.com/ru/company/otus/blog/659657/,kmoseenk
41,sql,2022-04-07 08:56,Яндекс Практикум запускает курс «SQL для работ...,https://habr.com/ru/company/yandex_praktikum/n...,eshulyndina
36,python,2022-03-18 15:31,24 марта Слёрм проведёт открытый урок «Первый ...,https://habr.com/ru/company/southbridge/news/t...,edeshina
62,big data,2022-03-10 08:30,10—24 марта: Big Data Dev Week от билайна,https://habr.com/ru/company/beeline/news/t/654...,Bee_brightside
25,python,2022-03-08 09:13,Вышел мартовский релиз расширения Python для V...,https://habr.com/ru/news/t/654707/,maybe_elf
75,big data,2022-02-18 12:51,Citymobil Data Meetup №7,https://habr.com/ru/company/citymobil/news/t/6...,leleles


Для каждого поискового слова скрипт срабатывает за 2 секунды

#### Задание 2  
Функция из обязательной части задания должна быть расширена следующим образом:

кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента 4 необходимо получить материалы с первых 4 страниц результатов;
в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:
<дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>

In [7]:
req = ['большие данные', 'python', 'big data']

Добавим еще две вспомогательные функции. 
1) Для определения количества страниц поиска (это максимальное количество, дольше поиск работать не будет)  
2) Для получения полного текста статьи и количества оценок

In [8]:
def get_num_pagins(url, word):
    '''Фукнция принимает на вход url и поисковое слово.
    Переходит на первую страницу поиска и 
    возвращает количе страниц
    '''
    params = {'q': word}
    html = get_html(url, params=params)
    soup = BeautifulSoup(html, 'html.parser')
    num = int(soup.find_all('div', 'tm-pagination__page-group')[-1].get_text())
    return num

In [9]:
get_num_pagins('https://habr.com/ru/search/', 'python')

50

In [10]:
def get_article_details(url):
    '''
    Фунция принимет на вход url конкретной статьи.
    Возвращает кортеж из двух значений:
    - число баллов (может быть и отрицательным)
    - текст новости и накопленное 
    '''
    html = get_html(url)
    soup = BeautifulSoup(html, 'html.parser')
    votes = soup.find('div', class_="tm-votes-meter tm-article-rating__votes-switcher").find('span').text    
    text = soup.find('div', class_='tm-article-body').get_text().strip()
    content = ''.join(re.findall(r'\S*\ ', text, flags=re.IGNORECASE)) # очищаем текст от переносов и табуляций
    return (int(votes), content)

In [11]:
get_article_details('https://habr.com/ru/news/t/497474/')

(13,
 'В рамках развития программы Microsoft AI for Earth в компании анонсировали новые этапы информационно-технические этапы по сохранению биоразнообразия и природных экосистем нашей планеты. 15 апреля 2020 года Microsoft объявила, что скоро запустит открытую вычислительную платформу под названием «Планетарный компьютер» для сбора, хранения и анализа данных о состоянии Земли. Причем доступ к платформе как для загрузки данных, так и для получения информации о состоянии Земли, например, изменении размеров лесных массивов, оценки рисков затоплений, землетрясений и других природных катастроф, бесплатно получат исследователи, экологи, ученые, специалисты по охране природы и окружающего мира, некоммерческие организации и государственные учреждения всего Microsoft AI for Earth — это часть глобального проекта компании под названием AI for Good, направленного на применение технологий искусственного интеллекта для борьбы с тремя глобальными проблемами: загрязнением окружающей среды (AI for Eart

In [12]:
def get_content_pro(req, pages=4):
    '''
    Функия получения расширенной таблицы со статьями по списку поисковых слов
    На вход принимает список и количество страниц поиска
    Внутри, 
    1) для каждого слова делает запрос к сайту
    2) перебирает блоки со статьями на первой странице поиска
    3) для каждой статьи собирает дату, название, ссылку, атора
    4) переходит по сстылке статьи и забирает оттуда оценки и полный текст
    5) добавляет конкретное поисковое слово
    6) доавляет полученную строку к датафрейму
    7) пробегает по всем страницам. если передать больше чем надо, ничего не вернется
    8) пробегает циклом по всем словам из переданного списка
    9) удаляет дубли по столбцу со ссылками
    '''
    url = 'https://habr.com/ru/search/'
    data = pd.DataFrame()
    for word in req:
        params = {'q': word}
        if pages <= get_num_pagins(url, word):
            for page in range(1, pages+1):
                html = get_html(url+f'page{page}/', params=params)
                soup = BeautifulSoup(html, 'html.parser')
                articles = soup.find('div', class_='tm-articles-list').find_all('div', 'tm-article-snippet')
                for item in tqdm(articles):
                    sleep(0.1)
                    date = pd.to_datetime(item.find('time').get('datetime')).strftime('%Y-%m-%d %H:%M') 
                    title = item.find('a', 'tm-article-snippet__title-link').text
                    link = 'https://habr.com' + item.find('a', 'tm-article-snippet__title-link').get('href')
                    try:
                        votes = get_article_details(link)[0]
                    except:
                        votes = None
                    try:
                        text = get_article_details(link)[1]
                    except:
                        text = None
                    author = item.find('span', 'tm-user-info__user').text.strip()
                    row = {'word':word, 
                           'date':date, 
                           'title':title, 
                           'link':link, 
                           'author':author, 
                           'votes':votes,
                           'text':text}
                    data = data.append(row, ignore_index=True)
    data.drop_duplicates(subset='link', inplace=True)
    return data

In [13]:
%%time
content_pro = get_content_pro(req, 5)

100%|██████████| 20/20 [00:18<00:00,  1.09it/s]

CPU times: user 41.8 s, sys: 1.22 s, total: 43 s
Wall time: 4min 26s


In [14]:
content_pro.sort_values(by='date', ascending=False).head(10)

,word,date,title,link,author,votes,text
127,python,2022-05-27 10:49,"«Ваша сезонность, сэр!»: ищем тренд и прогнози...",https://habr.com/ru/post/668186/,kaza4ka,6.0,"Как вы можете помнить по первой статье ""Маркет..."
126,python,2022-05-27 07:48,Работа с фреймворками Python: преимущества и п...,https://habr.com/ru/company/southbridge/blog/6...,edeshina,10.0,Фреймворки помогают ускорить разработку и сдел...
122,python,2022-05-24 07:57,Дайджест Слёрма: тест на уровень кунг-фу по Py...,https://habr.com/ru/company/southbridge/news/t...,Lika_Chernigo,9.0,Сделали для вас подборку свежих статей и выгод...
114,python,2022-05-18 15:33,Слёрм запускает 3-дневный интенсив по Python д...,https://habr.com/ru/company/southbridge/news/t...,edeshina,5.0,24-26 июня пройдёт онлайн-интенсив для инженер...
86,большие данные,2022-05-06 08:00,Конференция Data Fusion: большие спецы по боль...,https://habr.com/ru/company/vtb/blog/664596/,VTB,3.0,В 2022 году «бигдатой» никого не удивишь. Эта ...
213,big data,2022-05-04 08:44,Data-Science-процессы: Jupyter Notebook для пр...,https://habr.com/ru/company/vk/blog/662734/,Olga_Mokshina,37.0,Jovian Blues by Рефакторинг написанного в Note...
290,big data,2022-04-28 12:38,Big Data и логистика: чем большие данные полез...,https://habr.com/ru/post/663470/,ibm,-1.0,"Привет, Хабр! Мы – сервис для оптимизации внут..."
116,python,2022-04-22 11:42,TechnoMeetsPython. Онлайн митап о Python-разра...,https://habr.com/ru/news/t/662437/,technokratiya,2.0,27 апреля в 18:00 собираем питонистов на YouTu...
129,python,2022-04-18 03:29,Онлайн-митап от руководителя практики Python U...,https://habr.com/ru/company/usetech/news/t/661...,Usetech,0.0,"В конце марта Мстислав Казаков, руководитель п..."
13,большие данные,2022-04-07 14:23,17 лучших инструментов и технологий для работы...,https://habr.com/ru/company/otus/blog/659657/,kmoseenk,7.0,Мир больших данных становится только еще больш...


По логике скрипт лучше запускать по расписанию ночью. 50 страниц поиска должны выгрузиться за примерно 14-15 мин, следовательно 4 слова за 1 час, следовательно за 4 часа можно обработать 16 слов. Результат можно сохранить в файл и отправить на почту